In [33]:
import pandas as pd
import pyproj
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go

import geopandas as gpd

from ipywidgets import *
from ipyleaflet import *
!jupyter nbextension enable --py --sys-prefix ipyleaflet


Enabling notebook extension jupyter-leaflet/extension...
      - Validating: ok


In [22]:
Heads = pd.read_csv(str(Path('..') / 'Data' /'Well_Headers.csv'))

In [23]:
#The Petrel project says the CRS is GDA94, I have converted to D M S to decimal (Lat and Lon columns)
Heads

,Name,X,Y,Lat_D,Lat_M,Lat_S,Lat,Lon_D,Lon_M,Lon_S,Lon,DFE,TD_(TVDSS),TD_(MD)
0,Proteus_1,428152.83,8481148.02,-13,44,16.8752,-13.738021,122,20,7.6231,122.335451,21.8,5223.6,5249.7
1,Kronons_1,412599.26,8485490.73,-13,41,53.9802,-13.698328,122,11,30.2352,122.191732,21.8,5300.5,5329.0
2,Boreas_1,424077.28,8490107.52,-13,39,24.8717,-13.656909,122,17,52.7873,122.297996,21.8,5184.1,5210.0
3,Poseidon_1,425055.47,8493875.26,-13,37,22.3242,-13.622868,122,18,25.7039,122.307140,26.4,5070.3,5112.0
4,Pharos_1,435010.57,8487361.39,-13,40,55.2145,-13.682004,122,23,56.4780,122.399022,22.1,5197.3,5220.3
5,Poseidon_2,421661.83,8488800.77,-13,40,7.1748,-13.668660,122,16,32.2598,122.275628,21.8,5331.7,5356.0
6,Poseidon_North,428611.91,8499338.27,-13,34,24.8249,-13.573562,122,20,24.5557,122.340154,22.0,5263.3,5287.5
7,Torosa_1,399434.90,8466761.60,-13,52,2.0264,-13.867230,122,4,9.5897,122.069330,22.9,4645.4,4671.9


In [24]:

fig = px.scatter_geo(Heads, lat='Lat', lon='Lon', text= 'Name',               
                     )
fig.show()

In [36]:
#Below is an attempt to turn the dataframe into a Geodataframe, add to iphython leaflet map and use an iphython widget drop down to select


gdf = gpd.GeoDataFrame(
    Heads, geometry=gpd.points_from_xy(Heads.Lon, Heads.Lat))

In [37]:
gdf

,Name,X,Y,Lat_D,Lat_M,Lat_S,Lat,Lon_D,Lon_M,Lon_S,Lon,DFE,TD_(TVDSS),TD_(MD),geometry
0,Proteus_1,428152.83,8481148.02,-13,44,16.8752,-13.738021,122,20,7.6231,122.335451,21.8,5223.6,5249.7,POINT (122.335450861 -13.7380209)
1,Kronons_1,412599.26,8485490.73,-13,41,53.9802,-13.698328,122,11,30.2352,122.191732,21.8,5300.5,5329.0,POINT (122.191732 -13.6983278)
2,Boreas_1,424077.28,8490107.52,-13,39,24.8717,-13.656909,122,17,52.7873,122.297996,21.8,5184.1,5210.0,POINT (122.297996472 -13.6569088)
3,Poseidon_1,425055.47,8493875.26,-13,37,22.3242,-13.622868,122,18,25.7039,122.307140,26.4,5070.3,5112.0,POINT (122.307139972 -13.6228678)
4,Pharos_1,435010.57,8487361.39,-13,40,55.2145,-13.682004,122,23,56.4780,122.399022,22.1,5197.3,5220.3,POINT (122.399021667 -13.682004)
5,Poseidon_2,421661.83,8488800.77,-13,40,7.1748,-13.668660,122,16,32.2598,122.275628,21.8,5331.7,5356.0,POINT (122.275627722 -13.6686597)
6,Poseidon_North,428611.91,8499338.27,-13,34,24.8249,-13.573562,122,20,24.5557,122.340154,22.0,5263.3,5287.5,POINT (122.340154361 -13.5735625)
7,Torosa_1,399434.90,8466761.60,-13,52,2.0264,-13.867230,122,4,9.5897,122.069330,22.9,4645.4,4671.9,POINT (122.069330472 -13.8672296)


In [67]:
# The iphython widget below can be used as data selector in other functions


@interact( Well_select= widgets.Dropdown(options=Heads['Name'],value='Proteus_1',description='Well name:',disabled=False,))

#Ipython leaflet map that gets which well to plot from the drop down

def plot_wells(Well_select):
    
    defaultLayout = Layout(width='1024px', height='600px')
    
    m = Map(center=(28, -89), zoom=6, basemap=basemaps.Esri.NatGeoWorldMap, layout=defaultLayout)
    
    data_filter = gdf.loc[gdf['Name']== Well_select]
    
    
    center_x = data_filter.Lon.mean()
    center_y = data_filter.Lat.mean()
    
    m = Map(center=(center_y, center_x), zoom=7, basemap=basemaps.Esri.NatGeoWorldMap, layout=defaultLayout)
    
    geo_data = GeoData(geo_dataframe = data_filter,
    #geo_data_2= GeoData(geo_dataframe = gdf, style={'Color':'blue'})
    
    m.add_layer(geo_data)
    #m.add_layer(geo_data_2)
    m.add_control(FullScreenControl())
    
    measure = MeasureControl(
    position='topright',
    active_color = 'orange',
    primary_length_unit = 'miles'
    )
    m.add_control(measure)

    measure.completed_color = 'red'

    measure.add_length_unit('miles', 1.09361, 4)
    measure.secondary_length_unit = 'miles'

    measure.add_area_unit('sqmiles', 1.19599, 4)
    measure.secondary_area_unit = 'sqmiles'
    
    zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=7)
    jslink((zoom_slider, 'value'), (m, 'zoom'))
    widget_control1 = WidgetControl(widget=zoom_slider, position='bottomright')
    m.add_control(widget_control1)
    
   
    return m
    


interactive(children=(Dropdown(description='Well name:', options=('Proteus_1', 'Kronons_1', 'Boreas_1', 'Posei…